In [ ]:
#This is a code cell in a Jupyter notebook
print("Hello, world!")

Hello, world!


In [6]:
from sklearn.datasets import load_sample_images
import tensorflow as tf

images = load_sample_images().images
images = tf.keras.layers.CenterCrop(height=70, width=120)(images)
images = tf.keras.layers.Rescaling(scale=1/255)(images)

In [7]:
images.shape

TensorShape([2, 70, 120, 3])

In [13]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7, padding='same')
fmaps = conv_layer(images)

In [14]:
fmaps.shape

TensorShape([2, 70, 120, 32])

In [15]:
kernels, bianas = conv_layer.get_weights()
kernels.shape

(7, 7, 3, 32)

In [16]:
bianas.shape

(32,)

In [17]:
maxpool2d = tf.keras.layers.MaxPool2D(pool_size=2)

In [18]:
class DepthPool(tf.keras.layers.Layer):
def __init__(self, pool_size=2, **kwargs):
          super().__init__(**kwargs)
          self.pool_size = pool_size
def call(self, inputs):
shape = tf.shape(inputs) # shape[-1] is the number of
channels
groups = shape[-1] // self.pool_size # number of channel
groups
          new_shape = tf.concat([shape[:-1], [groups,
  self.pool_size]], axis=0)
return tf.reduce_max(tf.reshape(inputs, new_shape), axis=-1)